In [ ]:
import uproot
import awkward as ak

import numpy as np

import json

import time

# Download the files

## lumi file

https://opendata.cern.ch/record/14220

https://opendata.cern.ch/record/14220/files/Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt

## Data file

https://opendata.cern.ch/record/30522

I grabbed this one because it's smaller and easier to prototype with

4E3A26DE-E53B-A844-8048-36376617AE8D.root

Grabbed this one for testing with larger files

3B20EB8F-4FD1-D041-9513-1A82351756E1.root



In [ ]:
!wget https://opendata.cern.ch/record/14220/files/Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt

In [ ]:

!wget --no-check-certificate https://eospublic.cern.ch//eos/opendata/cms/Run2016G/DoubleMuon/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/4E3A26DE-E53B-A844-8048-36376617AE8D.root


!wget --no-check-certificate https://eospublic.cern.ch//eos/opendata/cms/Run2016G/DoubleMuon/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/3B20EB8F-4FD1-D041-9513-1A82351756E1.root



In [ ]:
#lumi_info = json.load(open('Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt'))
good_luminosity_sections = ak.from_json(open('Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt', 'rb'))

good_luminosity_sections['273158']

In [ ]:
#f = uproot.open('4E3A26DE-E53B-A844-8048-36376617AE8D.root')
f = uproot.open('3B20EB8F-4FD1-D041-9513-1A82351756E1.root')


t = f['Events']

#t.keys()

# Slow test

In [ ]:
t.num_entries

In [ ]:
nevents = t.num_entries
print(f"nevents: {nevents}")

In [ ]:
mask = np.zeros(nevents, dtype=int)
mask

In [ ]:
run = t['run'].array()
lumiBlock = t['luminosityBlock'].array()

print(run[0:10])
print(lumiBlock[0:10])

In [ ]:
print(len(run), len(lumiBlock))

start = time.time()

for idx,(r,lb) in enumerate(zip(run,lumiBlock)):

    if idx%10000==0:
        print(idx)

    #print(r,lb,idx)
    
    # r is an integer but appears as a string in the Awkward array 
    # of good luminosity sections
    good_blocks = good_luminosity_sections[str(r)]
    #print(good_blocks)

    # For debugging
    if idx<0:
        print(r,lb,idx,good_blocks)

    for good_block in good_blocks:
        if lb>=good_block[0] and lb<=good_block[1]:
            mask[idx] = 1

print(f"Time to run: {time.time() - start:.4f} seconds")

mask = mask.astype(bool)
mask

In [ ]:
mask[-1] = 1

In [ ]:
t['Muon_charge'].array()[mask]

# Faster?

In [ ]:
def build_lumi_mask(lumifile, tree, verbose=False):
    # lumifile should be the name/path of the file
    good_luminosity_sections = ak.from_json(open(lumifile, 'rb'))

    # Pull out the good runs as integers
    good_runs = np.array(good_luminosity_sections.fields).astype(int)
    #good_runs

    # Get the good blocks as an awkward array
    # First loop over to get them as a list
    all_good_blocks = []
    for field in good_luminosity_sections.fields:
        all_good_blocks.append(good_luminosity_sections[field])

    # Turn the list into an awkward array
    all_good_blocks = ak.Array(all_good_blocks)
    all_good_blocks[11]

    # Assume that tree is a NanoAOD Events tree
    nevents = tree.num_entries
    if verbose:
        print(f"nevents: {nevents}")
        print()
        print("All good runs")
        print(good_runs)
        print()
        print("All good blocks")
        print(all_good_blocks)
        print()

    # Get the runs and luminosity blocks from the tree
    run = t['run'].array()
    lumiBlock = t['luminosityBlock'].array()

    if verbose:
        print("Runs from the tree")
        print(run)
        print()
        print("Luminosity blocks from the tree")
        print(lumiBlock)
        print()

    # ChatGPT helped me with this part!
    # Find index of values in arr2 if those values appear in arr1

    def find_indices(arr1, arr2):
        index_map = {value: index for index, value in enumerate(arr1)}
        return [index_map.get(value, -1) for value in arr2]

    # Get the indices that say where the good runs are in the lumi file
    # for the runs that appear in the tree
    good_runs_indices = find_indices(good_runs, run)

    # For each event, calculate the difference between the luminosity block for that event
    # and the good luminosity blocks for that run for that event
    diff = lumiBlock - all_good_blocks[result]

    if verbose:
        print("difference between event lumi blocks and the good lumi blocks")
        print(diff)
        print()

    # If the lumi block appears between any of those good block numbers, 
    # then one difference will be positive and the other will be negative
    # 
    # If it it outside of the range, both differences will be positive or 
    # both negative.
    #
    # The product will be negagive if the lumi block is in the range
    # and positive if it is not in the range
    prod_diff = ak.prod(diff, axis=2)

    if verbose:
        print("product of the differences")
        print(prod_diff)
        print()

    mask = ak.any(prod_diff<=0, axis=1)

    return mask
    

    

    

In [ ]:
mask_test = build_lumi_mask('Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt', t)#, verbose=True)

In [ ]:
num = ak.num(t['Electron_eta'].array(), axis=0)
print(f"no mask: {num}")

num = ak.num(t['Electron_eta'].array()[mask_test], axis=0)
print(f"no mask: {num}")

In [ ]:
good_runs = np.array(good_luminosity_sections.fields).astype(int)
good_runs

In [ ]:
good_luminosity_sections

In [ ]:
all_good_blocks = []
for field in good_luminosity_sections.fields:
    all_good_blocks.append(good_luminosity_sections[field])

all_good_blocks = ak.Array(all_good_blocks)
all_good_blocks[11]

In [ ]:
# From ChatGPT

# Find index of values in arr2 if those values appear in arr1

def find_indices(arr1, arr2):
    index_map = {value: index for index, value in enumerate(arr1)}
    return [index_map.get(value, -1) for value in arr2]



In [ ]:
result = find_indices(good_runs, run)

#print(result)
print(len(run))
print(len(result))
print()

print(result[10])
print(run[10])

print(good_runs[272])
#lumi_runs

In [ ]:
good_runs[result]

In [ ]:
all_good_blocks[result]

#result

In [ ]:
diff = lumiBlock - all_good_blocks[result]

In [ ]:
diff[0:5]

In [ ]:
prod_diff = ak.prod(diff, axis=2)

prod_diff

In [ ]:
prod_diff<0

In [ ]:
mask2 = ak.any(prod_diff<=0, axis=1)

mask2

In [ ]:
ak.num(t['Electron_eta'].array(), axis=0)

In [ ]:
ak.num(t['Electron_eta'].array()[mask2], axis=0)

In [ ]:
ak.num(t['Electron_eta'].array()[mask], axis=0)

In [ ]:
t['Electron_eta'].array()[mask2]

# Tests

In [ ]:
mock_lumi_info = {}
mock_lumi_info[10] = [[1,100]]
mock_lumi_info[20] = [[2,50], [500,942]]
mock_lumi_info[30] = []
mock_lumi_info[40] = [[200,500], [520,594], [720,890]]

mock_lumi_info

In [ ]:
run = 40
#lp = 550 # lumi period
lp = 800

blocks = mock_lumi_info[run]
blocks = np.array(blocks)

print(blocks)
print()

dl0 = lp - blocks.T[0]
dl1 = blocks.T[1] - lp

print(dl0)
print(dl1)
print()

prod = dl0*dl1
test = (dl1>0) & (dl0>0)

print(dl0>0)

print(prod)
print(test)

In [ ]:
runs = np.array([10, 10, 10, 30, 40, 40])

print(runs)

In [ ]:
#keys = np.array(mock_lumi_info.keys())
keys = mock_lumi_info.keys()

print(keys)
print(type(keys))


#blocks = keys.tolist().index(runs)
#print(blocks)

In [ ]:
x = np.array(list(keys))

print(x)
print(type(x))

idx = np.where(x==40)

print(idx)

In [ ]:
x = ak.Array(keys)

ak.where(x==30)

In [ ]:
#ak.local_index(x, runs)

In [ ]:
? list.index

In [ ]:
? ak.index

In [ ]:
akindex = ak.index

In [ ]:
akindex.Index

In [ ]:
# From ChatGPT

def find_indices(arr1, arr2):
    index_map = {value: index for index, value in enumerate(arr1)}
    return [index_map.get(value, -1) for value in arr2]



In [ ]:
arr1 = np.array([10, 40, 50, 60])
arr2 = np.array([40, 40, 40, 60, 50])

result = find_indices(arr1, arr2)
print(result)

In [ ]:
lumi_runs = np.array(lumi_info.fields).astype(int)
print(lumi_runs[0:10])

runs = t['run'].array().to_numpy()
print(runs[0:10])

In [ ]:
result = find_indices(lumi_runs, runs)
#print(result)
print(len(runs))
print(len(result))
print()

print(result[10])
print(runs[10])

print(lumi_runs[272])
#lumi_runs

In [ ]:
lumi_runs[result]

In [ ]:
#lumi_info[result]
subset = lumi_info[['273158', '273302', '273302']]

print(type(subset))
